## Library Imports

In [ ]:
import sys
!{sys.executable} -m pip install numpy
!{sys.executable} -m pip install pandas
!{sys.executable} -m pip install python-dotenv
!{sys.executable} -m pip install requests
!{sys.executable} -m pip install xlsxwriter

In [ ]:
from dotenv import load_dotenv
import math
import numpy
import os
import pandas
import requests 
import xlsxwriter

load_dotenv()

## List of Stocks

In [ ]:
stocks =  pandas.read_csv('sp_500_stocks.csv')
stocks

## Acquiring an API Token

It will use IEx Cloud API.

In [ ]:
key = os.getenv("ACCESS_KEY")

## Marking first API Call

In [ ]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote/?token={key}'
data = requests.get(api_url).json()

## Parsing API Result

In [ ]:
price = data['latestPrice']
market_cap = data['marketCap'] / 1000000000000

## Adding Stocks to Pandas Dataframe

### Creating result Dataframe

In [ ]:
data_columns = ['Ticker', 'Stock Price', 'Market Capitalization', 'Number of Shares to Buy']
resultDataFrame = pandas.DataFrame(columns = data_columns)

### Append The first row into the DataFrame

In [ ]:
resultDataFrame.append(
    pandas.Series([
        symbol,
        price,
        market_cap,
        'N/A'
    ], index= data_columns),
    ignore_index=True
)

### Loopint through the Ticker of the Stocks

In [ ]:
resultDataFrame = pandas.DataFrame(columns = data_columns)
for stock in stocks['Ticker'][:5]:
    api_url = f'https://sandbox.iexapis.com/stable/stock/{stock}/quote/?token={key}'
    data = requests.get(api_url).json()
    resultDataFrame = resultDataFrame.append(
        pandas.Series([
            stock,
            data['latestPrice'],
            data['marketCap'] / 1000000000000,
            'N/A'
        ], index= data_columns),
        ignore_index=True
    )

resultDataFrame

### Using Batch API

In [ ]:
def chuncks(lst, n):
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [ ]:
symbol_groups = list(chuncks(stocks['Ticker'], 100))

In [ ]:
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))

In [ ]:
resultDataFrame = pandas.DataFrame(columns = data_columns)
for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote&token={key}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        resultDataFrame = resultDataFrame.append(
            pandas.Series([
                symbol,
                data[symbol]['quote']['latestPrice'],
                data[symbol]['quote']['marketCap'] / 1000000000000,
                'N/A'
            ], index= data_columns),
            ignore_index=True
        )

resultDataFrame

## Calculating the Number of Shares to Buy

In [ ]:
portfolio_size = input('Enter the value of you portfolio')
try:
    val = float(portfolio_size)
except ValueError:
    print('portfolio_size should be a number')
    portfolio_size = input('Enter the value of you portfolio')
    val = float(portfolio_size)

In [ ]:
position_size = val / len(resultDataFrame.index)

In [ ]:
for i in range(0, len(resultDataFrame.index)):
    stockPrice = resultDataFrame.loc[i, 'Stock Price']
    resultDataFrame.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / stockPrice)

resultDataFrame

## Formating to Excel

In [ ]:
writer = pandas.ExcelWriter('recommended trades.xlsx', engine = 'xlsxwriter')
resultDataFrame.to_excel(writer, 'Recommended Trades', index = False)

background_color = '#0E0A23'
font_color = '#FFFFFF'

string_format = writer.book.add_format({
    'font_color': font_color,
    'bg_color': background_color,
    'border': 1
})

dollar_format = writer.book.add_format({
    'num_format': '$ 0.00',
    'font_color': font_color,
    'bg_color': background_color,
    'border': 1
})

int_format = writer.book.add_format({
    'num_format': '0',
    'font_color': font_color,
    'bg_color': background_color,
    'border': 1
})

column_formats = {
    'A': ['Ticker', string_format],
    'B': ['Stock Price', dollar_format],
    'C': ['Market Capitalization', dollar_format],
    'D': ['Number of Shares to Buy', int_format]
}

for column in column_formats.keys():
    writer.sheets['Recommended Trades'].set_column(f'{column}:{column}', 18, column_formats[column][1])
    writer.sheets['Recommended Trades'].write(f'{column}1', column_formats[column][0], string_format)

writer.save()